In [1]:
import numpy as np
import pImpactR as impact
from copy import deepcopy as copy
import os
np.set_printoptions(precision=3)

In [2]:
ke,mass,freq = 2500000.0, 938272045.0, 30000000.0 
betx,emitx = 1.273, 1.461e-07
NL_c,NL_t = 0.01, 0.4

# Prepare Beam and Lattice

In [3]:
beam=impact.getBeam()
beam.nCore_y=1
beam.nCore_z=1
beam.n_particles=10
beam.current=0.0
beam.kinetic_energy = ke
beam.mass = mass
beam.frequency = freq
beam.distribution.distribution_type = 'Waterbag'
beam.distribution.betx = betx
beam.distribution.bety = betx
beam.distribution.emitx = emitx
beam.distribution.emity = emitx
beam.distribution.betz = 1265822570.9485857
beam.distribution.emitz = 1.2658226331473231e-05
beam.mesh.fld_solver = 'Smooth_Focusing'

lattice = [impact.getElem('loop'),
           impact.getElem('write_raw_ptcl'),
           0,
           impact.getElem('write_raw_ptcl'),
           0,
           impact.getElem('write_raw_ptcl')]
lattice[0].nturn = 1
lattice[1].file_id = 1001
lattice[3].file_id = 1002
lattice[5].file_id = 1003

# 1. SmoothFocusing NL

In [4]:
elemF = impact.getElem('nonlinear_insert_smooth_focusing')
elemF.transverse_scale_c = NL_c
elemF.strength_t = NL_t
elemF.betx = betx
elemF.length = 1.8
elemF.n_sckick = 50
elemF.n_map = 5

elemB = copy(elemF)
elemB.length =-elemF.length

lattice[2] = elemF
lattice[4] = elemB

impact.writeInputFile(beam,lattice);
impact.run()

0

In [5]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[-1.301e-18 -1.518e-18  8.132e-19 -3.469e-18  0.000e+00  0.000e+00]
 [-3.036e-18  4.879e-19 -1.518e-18 -6.505e-19  0.000e+00  0.000e+00]
 [ 2.168e-18 -3.036e-18  0.000e+00  4.066e-19  0.000e+00  0.000e+00]
 [ 6.505e-19 -5.421e-19  0.000e+00 -2.168e-19  0.000e+00  0.000e+00]
 [ 3.144e-18  1.301e-18  1.247e-18 -1.301e-18  0.000e+00  0.000e+00]
 [ 0.000e+00  1.355e-18  1.369e-18 -1.518e-18  0.000e+00  0.000e+00]
 [ 9.758e-19  8.674e-19 -3.253e-18 -6.505e-19 -2.842e-14  0.000e+00]
 [-2.711e-18 -2.819e-18 -2.277e-18  5.421e-19  0.000e+00  0.000e+00]
 [ 0.000e+00  2.277e-18 -2.602e-18  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  1.518e-18 -6.505e-19  1.626e-19  0.000e+00  0.000e+00]]


In [6]:
# print(pData1[:,:6]-pData0[:,:6])

In [7]:
cat test.in

!================= Beam & Control Parameters ================= 
1 1 
6 10 1 0 1 
32 32 32 9 0.014 0.014 0.1 
3 0 0 1 
10 
0.0 
1.0657889726792521e-09 
0.0010032631796813684 9.156210371571375e-05 0.0 1.0 1.0 0.0 0.0 
0.0010032631796813684 9.156210371571375e-05 0.0 1.0 1.0 0.0 0.0 
2.2092772151898714 1.065789e-10 0.0 1.0 1.0 0.0 0.0 
0.0 2500000.0 938272045.0 1.0 30000000.0 0.0 
!========================== Lattice ========================== 
0.0 0 0 -16 0.0 1 / 
0.0 0 1001 -2 1 1 / 
1.8 50 5 9 0.4 0.01 1.273 1.0 / 
0.0 0 1002 -2 1 1 / 
-1.8 50 5 9 0.4 0.01 1.273 1.0 / 
0.0 0 1003 -2 1 1 / 
